In [1]:
import numpy as np
from numpy.random import random as rand
import pandas as pd
from tqdm import tqdm 
import matplotlib.pyplot as plt

# 构建蝙蝠算法函数

In [36]:
# Parameters setting
# 蝙蝠数量 N_pop: population size, typically 10 to 40
# 学习的迭代次数 N_gen: number of generation
# 蝙蝠响度 A: loudness (constant or decreasing)
# 蝙蝠频度 r: pulse rate (constant or decreasing)
# This frequency range determines the scalings
# You should change these values if necessary
# 频率最小值 Qmin: frequency minmum
# 频率最大值 Qmax: frequency maxmum
# 每只蝙蝠的位置参数 d: number of dimensions
# 随机采样范围下限 lower: lower bound
# 随机采样范围上限 upper: upper bound

def bat_algorithm(objfun, N_pop=20, N_gen=1000, A=0.5, r=0.5,
    Qmin=0, Qmax=2, d=10, lower=-2, upper=2):
 
    N_iter = 0 # Total number of function evaluations
    # Limit bounds
    Lower_bound = lower * np.ones((1,d))#1*d，用来给蝙蝠d个维度设置参数的随机采样下界
    Upper_bound = upper * np.ones((1,d))#1*d，用来给蝙蝠d个维度设置参数的随机采样上界
 
    Q = np.zeros((N_pop, 1)) # 蝙蝠数量*1，蝙蝠目前的频率
    v = np.zeros((N_pop, d)) # 蝙蝠数量*d，蝙蝠目前的速度
    S = np.zeros((N_pop, d)) # 蝙蝠数量*d，蝙蝠目前的位置
 
    # 初始化蝙蝠种群 Initialize the population/soutions
    # Sol = np.random.uniform(Lower_bound, Upper_bound, (N_pop, d))
    # Fitness = objfun(Sol)
    Sol = np.zeros((N_pop, d))#蝙蝠数量*规模数量，初始化，表达通过随机参数的设定，拥有d个维度位置参数的蝙蝠
    Fitness = np.zeros((N_pop, 1))# 蝙蝠数量*1，初始化，表达最小误差情况下的值的函数，执行转换误差映射的是objfun函数，令其f()
    for i in range(N_pop):#再所有蝙蝠里对每一只蝙蝠迭代
        Sol[i] = np.random.uniform(Lower_bound, Upper_bound, (1, d))#1*规模数量，随机从上下界中内抽取随机数字，定义蝙蝠位置
        Fitness[i] = objfun(Sol[i])#将每次迭代出来的i行映射给objfun函数，赋值给Fitness的i行，Fitness衡量误差大小
 
    # 找到最佳的位置Find the initial best solution
    fmin = min(Fitness)#将Fitness一组数据中最小的值赋给fmin，fmin用来临时储存整个种群中最小误差值
    Index = list(Fitness).index(fmin)#将Fitness列表中最小值的位置赋值给Index
    best = Sol[Index]#将Sol中对应fmin的位置的那一行称作best，这一行是最初始蝙蝠的储存蝙蝠最佳位置参数
 
    # 开始迭代Start the iterations
    for t in tqdm(range(N_gen)):#循环N_gen次，N_gen为期望的学习次数
 
        # 循环所有蝙蝠 Loop over all bats/solutions
        for i in range(N_pop):#逐一对不同蝙蝠进行迭代
            # 含有随机变量的公式 Q[i] = Qmin + (Qmin - Qmax) * np.random.rand
            Q[i] = np.random.uniform(Qmin, Qmax)#定义i个的蝙蝠频率，为每一个蝙蝠设定的频率最大最小值中的一个随机数
            v[i] = v[i] + (Sol[i] - best) * Q[i]#更新i个蝙蝠速度
            S[i] = Sol[i] + v[i]#更新第i个蝙蝠的位置参数
            # ？？？？？？？Apply simple bounds/limits
            Sol[i] = simplebounds(Sol[i], Lower_bound, Upper_bound)#？？？？？？？
            # 蝙蝠频度 Pulse rate
            if rand() > r:#如果随机数大于给定的蝙蝠频度执行操作，如果没有跳过
                #用系数0.001限制随机游走 The factor 0.001 limits the step sizes of random walks
                S[i] = best + 0.001*np.random.randn(1, d)#用此公式更新目前第i个蝙蝠的目前位置，为best的位置+随机数
 
            # 从单个蝙蝠角度评估这一次BA迭代后的结论 Evaluate new solutions
            # print(i)
            Fnew = objfun(S[i])#将第i个蝙蝠的位置向量映射给objfun得到新的f(新位置),最终赋值给Fnew
            #如果蝙蝠位置改良了进那么进行第i个蝙蝠的位置更新 Update if the solution improves, or not too loud
            if (Fnew <= Fitness[i]) and (rand() < A):#如果新的位置确实变得更好了,即f(新位置)确实变小了
                Sol[i] = S[i]#用这一只蝙蝠新的位置取代旧的位置
                Fitness[i] = Fnew#用这一只蝙蝠新的f(新位置)取代旧的f(旧位置)
 
            # 从种群角度更新结论 update the current best solution
            if Fnew <= fmin:#如果这第i个蝙蝠的f(新位置)比原来种群中所有蝙蝠的f(位置)都要小的话做以下操作
                best = S[i]#将蝙蝠种群中的这第i个新的蝙蝠参数作为best
                fmin = Fnew#用这一只蝙蝠新的f(新位置)取代种群中的fmin，作为目前种群中的误差最小值
 
        N_iter = N_iter + N_pop#代表了进行了多少次的BA算法，每循环一次累加一次蝙蝠数量，例如，循环4次20只蝙蝠就是80
 
    print('Number of evaluations: ', N_iter)#打印BA的次数
    print("Best = ", best, '\n fmin = ', fmin)#打印最佳蝙蝠最佳的位置参数和最小的误差
 
    return best#此函数返回最优的蝙蝠位置参数
 
 
def simplebounds(s, Lower_bound, Upper_bound):#定义？？？？？？？
 
    Index = s > Lower_bound
    s = Index * s + ~Index * Lower_bound
    Index = s < Upper_bound
    s = Index * s + ~Index * Upper_bound
 
    return s
 
 
# u: array-like
def test_function(u):
    a = u ** 2
    return a.sum(axis=0) 

# 用普通函数测试BA模型

In [51]:
#利用简单的二次函数测试蝙蝠函数
def function_test1(x):
    T=(x[0]-1)**2
    return T
bat_algorithm(function_test1,N_pop=20, N_gen=2000,d=1,lower=-10, upper=10)

100%|██████████| 2000/2000 [00:01<00:00, 1848.66it/s]

Number of evaluations:  40000
Best =  [1.00008451] 
 fmin =  5.2607307202918235e-18


array([1.00008451])

In [52]:
def function_test2(x):
    y1 = 1 / 4000 * sum(np.power(x, 2))
    y2 = 1
    for h in range(x.size):
        y2 = y2 * np.cos(x[h] / np.sqrt(h + 1))
    y = y1 - y2 + 1
    return y
bat_algorithm(function_test2,N_pop=20, N_gen=2000,d=10,lower=-10, upper=10)

100%|██████████| 2000/2000 [00:02<00:00, 781.31it/s]

Number of evaluations:  40000
Best =  [-9.66847501e-04  4.43870531e+00 -5.42984437e+00 -2.33759132e-03
 -7.00621586e+00 -7.66998957e+00  1.48675162e-03 -8.84906855e+00
 -9.38786246e+00 -5.76273603e-03] 
 fmin =  0.08115899106281865


array([-9.66847501e-04,  4.43870531e+00, -5.42984437e+00, -2.33759132e-03,
       -7.00621586e+00, -7.66998957e+00,  1.48675162e-03, -8.84906855e+00,
       -9.38786246e+00, -5.76273603e-03])

In [59]:
def function_test3(x):
    sum1=np.cos(x-2)
    sum2=np.sin(x-2)
    y=sum1/sum2
    return y
bat_algorithm(function_test3,N_pop=200, N_gen=2000,d=1,lower=-10, upper=10)

100%|██████████| 2000/2000 [00:11<00:00, 175.50it/s]

Number of evaluations:  400000
Best =  [1.99972168] 
 fmin =  [-80162582.6638076]


array([1.99972168])

# 构建神经网络

In [38]:
#定义激活函数
def fp(x,p=5):
    exp1=4*p-2;exp2=1/(4*p-2)
    return x/(0.000001+(1+x**(exp1))**exp2)

#定义损失函数
def objfunction(beta,C=1):#beta是n*1
    sum1=sum2=0
    for i in range(beta.size):
        sum1+=beta[i]**2
    sqrt1=np.sqrt(sum1)
    error=H.dot(beta.T)-y
    for i in range(error.size):
        sum2+=error[i]**2
    sqrt2=np.sqrt(sum2)
    loss=0.5*sqrt1+sqrt2*C/2
    return loss

#  利用boston房价数据测试BA-BP模型预测能力

In [39]:
import numpy as np
import pandas as pd
data = pd.read_csv(r"dataset/boston.csv")
new_columns = data.columns.insert(0, "Intercept")
t= data.reindex(columns=new_columns, fill_value=1)
t=pd.DataFrame.drop(t,columns='Unnamed: 0')
X=t.iloc[:400,:-1]
y=t.iloc[:400, -1]
X=np.asarray(X) 
y=np.asarray(y)
X_test=t.iloc[400:,:-1]
y_test=t.iloc[400:, -1]
t.shape

(400, 14)

# 对于boston进行BA-BP测试

In [40]:
H = fp(X,p=1)#m*n

In [41]:
Best=bat_algorithm(objfunction,N_pop=20, N_gen=500,d=14, lower=-10, upper=10)
atemp=X_test.dot(Best)
atemp

100%|██████████| 500/500 [00:02<00:00, 196.99it/s]

Number of evaluations:  10000
Best =  [-1.86785594  6.44620712  4.48258377  4.05842359  2.14781617  0.67076186
  5.43051136  6.57668792 -1.19135346  4.08446179  6.46960902  2.01216067
 -4.72918409 -0.49998348] 
 fmin =  102.29609188230263


400    2731.687472
401    2665.312430
402    2732.109694
403    2698.101103
404    3050.582594
          ...     
501    -447.243624
502    -424.506825
503    -325.716769
504    -323.806646
505    -403.280063
Length: 106, dtype: float64

# 对于german.data-numeric数据样本数据处理

In [42]:
data=np.loadtxt(r"dataset/german.data-numeric")
data

array([[ 1.,  6.,  4., ...,  0.,  1.,  1.],
       [ 2., 48.,  2., ...,  0.,  1.,  2.],
       [ 4., 12.,  4., ...,  1.,  0.,  1.],
       ...,
       [ 4., 12.,  2., ...,  0.,  1.,  1.],
       [ 1., 45.,  2., ...,  0.,  1.,  2.],
       [ 2., 45.,  4., ...,  0.,  1.,  1.]])

In [43]:
#划分测试集和训练集
X =data[:850,:-1]
y = data[:850, -1]
X = np.asarray(X) 
y = np.asarray(y)
X_test=data[850:,:-1]
y_test=data[850:, -1]

# 对于german.data-numeric进行BA-BP测试

In [44]:
H = fp(X,p=1)#m*n
H.shape

(850, 24)

In [45]:
W_1=bat_algorithm(objfunction,N_pop=20, N_gen=500,d=24)

100%|██████████| 500/500 [00:05<00:00, 99.22it/s] 

Number of evaluations:  10000
Best =  [-0.07777947  0.32246224 -0.18287531 -0.2917735  -0.22691018  0.37245373
  0.32310731 -0.75806661  0.13240525 -0.28589038  0.52921066  0.58047997
  0.48853512 -0.58039325  1.54237839  0.28402395  0.15964946  0.32965744
 -0.71410388  0.39265821  0.13560941 -0.56817679 -0.58280861 -0.70892815] 
 fmin =  8.830769063844699


In [46]:
atemp=X.dot(W_1.T)
fp(atemp)

array([-0.99999995, -0.99999981, -0.99999992, -0.99999995, -0.99999995,
       -0.99999996, -0.99999993, -0.99999993, -0.99999995, -0.99999989,
       -0.99999965, -0.99999956, -0.99999973, -0.99999989, -0.9999998 ,
       -0.99999917, -0.99999992, -0.99999995, -0.99999997, -0.99999986,
       -0.99999993, -0.99999993, -0.99999991, -0.99999992, -0.9999998 ,
       -0.99999991, -0.99999986, -0.99999985, -0.99999991, -0.99999994,
       -0.9999999 , -0.99999988, -0.99999994, -0.99999993, -0.99999987,
       -0.99999966, -0.9999999 , -0.99999989, -0.99999987, -0.99999965,
       -0.99999952, -0.99999977, -0.99999996, -0.99999992, -0.99999994,
       -0.99999988, -0.99999975, -0.99999985, -0.99999996, -0.99999988,
       -0.99999984, -0.99999992, -0.99999972, -0.9999999 , -0.99999989,
       -0.99999981, -0.99999996, -0.99999996, -0.99999425, -0.99999992,
       -0.99999984, -0.99999992, -0.99999988, -0.99999997, -0.99999988,
       -0.99999994, -0.99999989, -0.99964311, -0.99999971, -0.99